####Converting dataset volumes in tables into bronze using streaming and DLT to make NRT flow (Ingest raw data into bronze -still raw-)


In [0]:
import dlt
from pyspark.sql.functions import *

In [0]:
input_path = "/Volumes/well_logs_nrt/landing/json_files_volume/"
schema_path = "/Volumes/well_logs_nrt/autoloader/schemas_volume/all_wells_schema"

@dlt.table(
    name = "all_wells_bronze",
    comment = "Table that contains all wells in bronze layer",
    table_properties = {"layer" : "bronze", "type" : "well log"}
)

def all_wells_bronze():
  return (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.schemaLocation", schema_path)
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns") # In case that appears new columns in the log
    # .option("multiLine", "true")                             # Enable it if your JSON are multiline
    # .option("rescuedDataColumn", "_rescued_data")            # Unnecessary: exists by default. Only if I want to rename it.
    # .option("cloudFiles.includeExistingFiles", "true")       # 1st boot ingers historical. I use it only if "false"
    .load(input_path)
    .withColumn("bronze_ingestion_time", current_timestamp())
    .withColumn("file_mod_time", col("_metadata.file_modification_time"))  # útil p/ auditoría
  )